# Install packages

In [ ]:
!pip install datasets
!pip install transformers
!pip install tqdm
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 22.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/si

# Import packages

In [ ]:
from datasets import load_dataset, load_metric, list_metrics
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollator, T5ForConditionalGeneration, T5TokenizerFast
import torch
from tqdm import tqdm
from typing import Dict, List, Optional
import dataclasses
from dataclasses import dataclass, field
import logging
import os
import sys
import numpy as np
import torch
import json
import datasets
from huggingface_hub import notebook_login
from transformers import (
    T5ForConditionalGeneration, 
    T5Tokenizer, 
    EvalPrediction,
    DataCollator,
    Trainer,
    TrainingArguments)
from google.colab import files

os.environ["WANDB_DISABLED"] = "true"

logger = datasets.logging.get_logger(__name__)

# Hugging Face login

In [ ]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Load SQuAD dataset

In [ ]:
files.upload()

rawDataset = load_dataset("t5_squad.py")

rawDataset["train"][0]

Saving t5_squad.py to t5_squad.py


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset t5_squad downloaded and prepared to /root/.cache/huggingface/datasets/t5_squad/plain_text/1.0.0/02ae0815e8483cc76579286179faeb8c8fdbdd328e6741f5c465d9b0bddb8a77. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'context': 'generate questions: Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'questions': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? {sep_token} What is in front of the Notre Dame Main Building? {sep_token} The Basilica of the Sacred heart at Notre Dame is beside to which structure? {sep_token} What is the Grotto

# Data preprocessing

In [ ]:
baseCheckpoint = "t5-base"

model = T5ForConditionalGeneration.from_pretrained(baseCheckpoint)
tokenizer = T5TokenizerFast.from_pretrained(baseCheckpoint)

tokenizer.sep_token = '<sep>'
tokenizer.add_tokens(['<sep>'])
tokenizer.sep_token_id

model.resize_token_embeddings(len(tokenizer))

maxInputLength =  512
maxTargetLength = 64

def convertDatasetToFeatures(dataset):

    inputEncodings = tokenizer.batch_encode_plus(dataset['context'], 
                                                  max_length=maxInputLength, 
                                                  add_special_tokens=True,
                                                  truncation=True, 
                                                  pad_to_max_length=True)
    
    targetEncodings = tokenizer.batch_encode_plus(dataset['questions'], 
                                                   max_length=maxTargetLength, 
                                                   add_special_tokens=True,
                                                   truncation=True, pad_to_max_length=True)
                                                   
    encodings = {
        'input_ids': inputEncodings['input_ids'], 
        'attention_mask': inputEncodings['attention_mask'],
        'decoder_input_ids': targetEncodings['input_ids']
        ,'decoder_attention_mask': targetEncodings['attention_mask']
    }

    return encodings

def addEOSToFeature(feature):
  feature['context'] = feature['context'] + " </s>"
  feature['questions'] = feature['questions'] + " </s>"
  return feature


def addSpecialTokensToFeatureQuestions(feature):
  feature['questions'] = feature['questions'].replace("{sep_token}", '<sep>')
  return feature


tokenizedDataset  = rawDataset.map(addEOSToFeature)
tokenizedDataset = tokenizedDataset.map(addSpecialTokensToFeatureQuestions)
tokenizedDataset  = tokenizedDataset.map(convertDatasetToFeatures,  batched=True)

tokenizedDataset["train"][0]["context"]

tokenizedDataset = tokenizedDataset.remove_columns(
    ["context", "questions"]
)

trainDataset = tokenizedDataset["train"]
validDataset = tokenizedDataset["validation"]

columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
trainDataset.set_format(type='torch', columns=columns)
validDataset.set_format(type='torch', columns=columns)

torch.save(trainDataset, 'train_data.pt')
torch.save(validDataset, 'valid_data.pt')

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Map:   0%|          | 0/18896 [00:00<?, ? examples/s]

Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

Map:   0%|          | 0/18896 [00:00<?, ? examples/s]

Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

Map:   0%|          | 0/18896 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Map:   0%|          | 0/2067 [00:00<?, ? examples/s]

# Fine tuning the T5 model and push to hub

In [ ]:
@dataclass
class T2TDataCollator():
  def __call__(self, batch: List) -> Dict[str, torch.Tensor]:
    inputIds = torch.stack([feature['input_ids'] for feature in batch])
    lmLabels = torch.stack([feature['decoder_input_ids'] for feature in batch])
    lmLabels[lmLabels[:, :] == 0] = -100 
    attentionMask = torch.stack([feature['attention_mask'] for feature in batch])
    decoderAttentionMask = torch.stack([feature['decoder_attention_mask'] for feature in batch])
    
    return {
      'input_ids': inputIds, 
      'attention_mask': attentionMask,
      'labels': lmLabels, 
      'decoder_attention_mask': decoderAttentionMask
    }


trainingArgs = TrainingArguments(output_dir="./out", 
  per_device_train_batch_size=4, 
  per_device_eval_batch_size=4,
  gradient_accumulation_steps=16,
  learning_rate=1e-4, 
  num_train_epochs=7,
  logging_steps=100,
  run_name="t5-simple-qg-eng",
  evaluation_strategy="steps",
  save_steps=500,
  report_to=None,
  push_to_hub=True,
  push_to_hub_model_id="t5-simple-qg-eng"
)

trainer = Trainer(
    model=model,
    args=trainingArgs,
    train_dataset=trainDataset,
    eval_dataset=validDataset,
    data_collator=T2TDataCollator()
)

trainer.train()

trainer.push_to_hub("t5-simple-qg-eng")

using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Cloning https://huggingface.co/mrbalazs5/t5-simple-qg-eng into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18896
  N

Step,Training Loss,Validation Loss
100,2.584000,1.910817
200,1.966400,1.727507
300,1.846600,1.663376
400,1.741200,1.638291
500,1.713400,1.620192
600,1.694000,1.604899
700,1.629700,1.597497
800,1.626100,1.593173
900,1.614900,1.587460
1000,1.569000,1.589304


***** Running Evaluation *****
  Num examples = 2067
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2067
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2067
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2067
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2067
  Batch size = 4
Saving model checkpoint to ./out/checkpoint-500
Configuration saved in ./out/checkpoint-500/config.json
Configuration saved in ./out/checkpoint-500/generation_config.json
Model weights saved in ./out/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2067
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2067
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2067
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2067
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2067
  Batch size = 4
Saving model checkpoint to ./out/checkpoint-1000
Configuration save

Upload file pytorch_model.bin:   0%|          | 32.0k/850M [00:00<?, ?B/s]

Upload file runs/Mar11_10-26-32_30219b4c28fe/events.out.tfevents.1678530398.30219b4c28fe.406.0: 100%|#########…

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/mrbalazs5/t5-simple-qg-eng
   c0a552c..e3aad99  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/mrbalazs5/t5-simple-qg-eng
   c0a552c..e3aad99  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'dataset': {'name': 't5_squad', 'type': 't5_squad', 'config': 'plain_text', 'split': 'validation', 'args': 'plain_text'}}
To https://huggingface.co/mrbalazs5/t5-simple-qg-eng
   e3aad99..5426d64  main -> main

   e3aad99..5426d64  main -> main



'https://huggingface.co/mrbalazs5/t5-simple-qg-eng/commit/e3aad99a54d9c70a208788f5c0795f7dd6e64b2c'